In [ ]:
# default_exp scrape6K

# scrape6K

> Scrape item summaries from 6-K SEC filings.

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
from nbdev import show_doc

In [ ]:
#export

import collections
import itertools
import os
import re

from secscan import utils, dailyList, basicInfo, infoScraper

default6KDir = os.path.join(utils.stockDataRoot,'scraped6K')

6K scraper class - scrape XXX from the SEC filing:

In [ ]:
#export

extract6KPats = [
    re.compile(r'12g.*?no(?:.{1,20}is marked.{1,100}82.)?(.*)signature',re.IGNORECASE),
    re.compile(r'101\s*\(b\)\s*\(7\)(?:.{1,20}note\s*:.*?on edgar.)?(.*)signature',re.IGNORECASE),
    re.compile(r'exhibit\s+index(.*)signature',re.IGNORECASE),
    re.compile(r'signature.*exhibit(?:\s+index)?(.*)',re.IGNORECASE),
]

def parse6K(accNo, formType=None, textLimit=500) :
    print(f'[{accNo}]',end=' ')
    info = basicInfo.getSecFormInfo(accNo, formType)
    mainText = utils.downloadSecUrl(info['links'][0][3], toFormat='souptext')
    for extract6KPat in extract6KPats :
        m = extract6KPat.search(mainText)
        if m :
            m = m.group(1).strip()
            if len(m)>20 :
                break
    if m and len(m)>20 :
        info['mainText'] = m[:textLimit]
    else :
        print('*** no main text')
    return info

class scraper6K(infoScraper.scraperBase) :
    def __init__(self, infoDir=default6KDir, startD=None, endD=None, fSuff='m.pkl', **pickle_kwargs) :
        super().__init__(infoDir, '6-K', startD=startD, endD=endD, fSuff=fSuff, **pickle_kwargs)
    def scrapeInfo(self, accNo, formType=None) :
        return parse6K(accNo, formType), None

Test 6-K scraper class:

In [ ]:
dl = dailyList.dailyList(startD='empty')
dl.updateForDays('20210701','20210705')
assert len(dl.getFilingsList(None,'6-K')[0])==188,"testing 6-K scraper class (daily list count)"

20210704 WEEKEND 20210703 WEEKEND 20210702 ### list index 13 filings for 20210702: 6569 * 20210701 filings for 20210701: 5573 * 

In [ ]:
# dailyList.dlCountFilings(startD='20210101',endD='20210331',formClass='6-K',noAmend=False)

5788

In [ ]:
# dl = dailyList.dailyList(startD='20210101',endD='20210331')
# l = dl.getFilingsList(formClass='6-K')[0]

In [ ]:
# i = basicInfo.getSecFormInfo(l[623][3])

In [ ]:
# [tup for tup in i['links'] if tup[2].startswith('EX-99')]

[('eh210139217_ex9901.htm',
  'EXHIBIT 99.1',
  'EX-99.1',
  '/Archives/edgar/data/886986/000095014221001036/eh210139217_ex9901.htm')]

In [ ]:
#hide
# uncomment and run to regenerate all library Python files
# from nbdev.export import notebook2script; notebook2script()